In [ ]:
# Loading Dependencies for Medical Image Analysis
import optuna
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt # Import matplotlib.pyplot
import torch.nn as nn
from torchsummary import summary
import torchvision.models as models
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import torchvision
import torch.optim as optim  # Import the optim module
import numpy as np
import torchvision.transforms as transforms
#from torch.utils.data import DataLoader, Subset
#from torchvision.datasets import ImageFolder
#from torchvision import datasets
import torchvision.datasets as datasets
from torch.utils.data import Dataset, DataLoader, Subset
#from torchvision import transforms
import os
import torch
from PIL import Image
#import matplotlib.pyplot as plt
#import time
import copy
#import os
import IPython
import time
from sklearn.model_selection import KFold
import threading
import optuna.visualization
import albumentations as A
from albumentations.pytorch import ToTensorV2
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import GPyOpt
import GPy
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from torch.optim.lr_scheduler import (
    StepLR,
    ReduceLROnPlateau,
    CosineAnnealingLR,
    CyclicLR,
    OneCycleLR,
)

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-09 07:15:28.794863: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-09 07:15:28.873179: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 2

In [ ]:
def create_scheduler(optimizer, scheduler_type, **kwargs): # This function defines the scheduler types
    if scheduler_type == "StepLR":
        return StepLR(optimizer, **kwargs)
    elif scheduler_type == "ReduceLROnPlateau":
        return ReduceLROnPlateau(optimizer, **kwargs)
    elif scheduler_type == "CosineAnnealingLR":
        return CosineAnnealingLR(optimizer, **kwargs)
    elif scheduler_type == "CyclicLR":
        return CyclicLR(optimizer, cycle_momentum=False, **kwargs)
    elif scheduler_type == "OneCycleLR":
        return OneCycleLR(optimizer, **kwargs)
    else:
        raise ValueError(f"Unknown scheduler type: {scheduler_type}")

In [ ]:
#scheduler_types = ["StepLR", "ReduceLROnPlateau", "CosineAnnealingLR", "CyclicLR", "OneCycleLR"]
scheduler_types = "OneCycleLR" # Specify the scheduler here

In [ ]:
K = 10  # Example K value
best_params_for_k = {
    "learning_rate":0.0001823817366005471, # Example learning rate for K=5
    "dropout_rate":0.21623344843587963,     # Example dropout rate for K=5
    "batch_size":16, # Example batch size for K=5
    "num_epochs":86 # Example number of epochs for K=5
}
#scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=0.0005537021899767878)  # Initialize scheduler


In [ ]:

def Model_ResNet101_PyTorch(learning_rate, dropout_rate):
    # Load pre-trained ResNet101
    model = models.resnet101(weights='ResNet101_Weights.DEFAULT')

    # Modify the input layer to accept 3 channels (assuming your input is RGB)
    # model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # Freeze all layers initially
    for param in model.parameters():
        param.requires_grad = False

    # Unfreeze the desired layers for fine-tuning
    for param in model.layer3.parameters(): # Unfreeze Layer 3
        param.requires_grad = True

    for param in model.layer4.parameters(): # Unfreeze Layer 4
        param.requires_grad = True

    for param in model.fc.parameters(): # Unfreeze the fully connected layer
        param.requires_grad = True

    # Modification of the classifier layer (fully connected layer)
    num_ftrs = model.fc.in_features
    model.fc = nn.Sequential(
        nn.Linear(num_ftrs, 3),        # 3 output classes (adjust if you have a different number of classes)
        nn.Dropout(dropout_rate),
        nn.Softmax(dim=1)              # Softmax activation for multi-class classification
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    summary(model, (3, 224, 224), device = str(device))  # Print model summary (adjust input size if needed)

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss() # Appropriate for multi-class classification

    # Define optimizer - consider using a lower learning rate for earlier unfrozen layers if needed
    optimizer = optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, weight_decay=0.01)


    return model, criterion, optimizer

In [ ]:
model, criterion, optimizer = Model_ResNet101_PyTorch(learning_rate=best_params_for_k["learning_rate"],
                                                      dropout_rate= best_params_for_k["dropout_rate"])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14          [-1, 256,

In [ ]:
# We pass the scheduler types to the training loop at this portion
#scheduler = ReduceLROnPlateau(optimizer, mode='min', factor =0.13700840866435451, patience = 10)
#scheduler = StepLR(optimizer, step_size = 7, gamma=0.6529691827085107)
#scheduler = CosineAnnealingLR(optimizer, T_max=6, eta_min=9.777678382795824e-05) # Initialize scheduler
#scheduler = OneCycleLR(optimizer, max_lr=0.005433329251927817, epochs=86, steps_per_epoch=19)
scheduler = CyclicLR(optimizer, base_lr = 2.5696458587770245e-06, max_lr = 0.000577228568316326, step_size_up =8.0,
                    step_size_down = 9.0, mode = "triangular")

In [ ]:
class MedicalImageDataset(Dataset):
       def __init__(self, root_dir, image_extensions, transform=None):
           self.root_dir = root_dir
           self.image_extensions = image_extensions
           #self.transform = transform
           self.image_paths = []
           self.labels = []

           for class_name in os.listdir(root_dir):
               class_path = os.path.join(root_dir, class_name)
               if os.path.isdir(class_path):
                   for filename in os.listdir(class_path):
                       if filename.lower().endswith(image_extensions):
                           self.image_paths.append(os.path.join(class_path, filename))
                           self.labels.append(class_name)  # Assuming subfolder name is the label

       def __len__(self):
           return len(self.image_paths)

       def __getitem__(self, idx):
           image_path = self.image_paths[idx]
           image = np.array(Image.open(image_path).convert('RGB'))  # Convert to NumPy array
           label = self.labels[idx]
           label_mapping = {'Benign': 0, 'Malignant': 1, 'Normal': 2}
           label = label_mapping.get(label)  # Apply the label mapping here
           #image = np.array(image)

           return image, label

In [ ]:
dataset_path = '/home/workspace/PetersWorkspace/Abbans/TMPFS_Medical_Images/Medical_Images/'
#dataset_path = '/usr/bin/Medical_Images'
image_extensions = ('.png','.jpg')

dataset = MedicalImageDataset(dataset_path, image_extensions)

In [ ]:
# Splitting the dataset based on their split proportions
Train_ratio = 0.70 # Ration of Training Data
Validaton_ratio = 0.15  # Ratio of Testing Data
Testing_ratio = 0.15 # Ration of Testing Data

torch.manual_seed(42)  # You can choose any integer as your seed
# Calculate the sizes of the train, validation, and test sets
Dataset_size = len(dataset) # Determines the dataset size
#Train_size = int(Train_ratio * Dataset_size) # Determines the train dataset size
#Validation_size = int(Validaton_ratio * Dataset_size) # Determines the validation dataset size
Test_size = int(Testing_ratio * Dataset_size) # Determines the test dataset size
#Test_size = Dataset_size - Train_size - Validation_size # Determines the test dataset size
Dataset_Train_Valid = Dataset_size - Test_size # Determines the new dataset size

# Split the dataset into train, validation, and test sets
 #Train_dataset, Validation_dataset, Test_dataset = torch.utils.data.random_split(dataset, [Train_size, Validation_size, Test_size])
Dataset_New, Test_dataset = torch.utils.data.random_split(dataset, [Dataset_Train_Valid, Test_size])

In [ ]:
#print(len(Train_dataset))
#print(len(Validation_dataset))
#print(len(Test_dataset))
print(len(Dataset_New))
print(len(Test_dataset))

661
116


In [ ]:
# Defining Augmentation for my Training Dataset
#For the training dataset
#For the training dataset
Fundamental_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(degrees=(-10, 10)),
    transforms.RandomAffine(degrees=0, translate=(0.08, 0.08), scale=(0.92, 1.08)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.05),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.85, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
Advanced_transform = A.Compose([
    A.MedianBlur(blur_limit=3, p=1),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=5, p=0.5),
    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.08, rotate_limit=12, p=0.5),
    A.RandomSizedCrop(min_max_height=(160, 224), size =(224, 224), p=0.5),
    A.ElasticTransform(alpha=0.8, sigma=40, alpha_affine=40, p = 0.5),
    A.GridDistortion(p=0.4, distort_limit=0.4),
    A.OpticalDistortion(p=0.05, distort_limit=0.05, shift_limit=0.5),
    #A.GaussianBlur(blur_limit=(3, 7), p=0.5),  # Gaussian Blur applied after spatial transformations
    # CoarseDropout(Random Erasing)
    A.CoarseDropout(max_holes=1, max_height=32, max_width=32, min_holes=1, min_height=16,
                    min_width=16, fill_value=0,
        p=0.5,),

    # Pixel-Level Transformations
    A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=0.3),
    #A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=20, p=0.4),
    A.CLAHE(clip_limit=3.0, tile_grid_size=(6, 6), p=0.25),  # Contrast Limited Adaptive Histogram Equalization

    # Noise Injection
    #A.GaussNoise(var_limit=(0.01, 0.05), p=0.5),
    #A.MultiplicativeNoise(multiplier=(0.9, 1.1), elementwise=True, p=0.3),  # Speckle noise

         # Blurring and Sharpening
    #A.OneOf([A.MedianBlur(blur_limit=3, p=0.5),],p=0.1),

    A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),

    # Normalization
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

In [ ]:
Train_transform = transforms.Compose([
    transforms.ToPILImage(), # Conversion of Images into PIL for Albumentation
    Advanced_transform, # Application of advanced transformations
    Fundamental_transform, # Application of fundamental transformation
])

In [ ]:
# Defining Augmentation for my Validaton Dataset
Validation_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Defining Augmentation for the Testing Dataset
Test_transform = transforms.Compose([
    # Only resize and normalize for test set
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
# Applying the Augmentation to the Test dataset
Test_dataset.dataset.transform = Test_transform

In [ ]:
Dataset_New.dataset.transform = Train_transform

In [ ]:
def my_collate_fn(batch):
            images = []
            labels = []

            for image, label in batch:
                image = transforms.ToPILImage()(image)  # Convert to PIL Image
                image = transforms.Resize((224, 224))(image)  # Add resize transform here
                image = transforms.ToTensor()(image)  # Convert to Tensor using ToTensor
                images.append(image)
                labels.append(label)

            # Stack images into a batch tensor
            images = torch.stack(images, dim=0)
            labels = torch.tensor(labels)
            return images, labels


In [ ]:
# Creating my Test dataloader
Test_dataloader = DataLoader(Test_dataset, batch_size=best_params_for_k["batch_size"],
                             shuffle=False, collate_fn = my_collate_fn, num_workers = 32) # New

In [ ]:
train_loader = DataLoader(Dataset_New, batch_size=best_params_for_k["batch_size"], shuffle=True, num_workers = 32, collate_fn = my_collate_fn)

In [ ]:
# Model to be runned for final evauation process:
def Train_Model_Final(model, criterion, optimizer, train_loader, num_epochs, scheduler, scheduler_type):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    train_acc_history = []
    train_loss_history = []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(str(device))

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Training phase only
        model.train()
        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(True):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                loss.backward()
                optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)

        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        train_acc_history.append(epoch_acc)
        train_loss_history.append(epoch_loss)

        # Update best model weights (optional, based on training accuracy)
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

        if scheduler_type in ["CosineAnnealingLR", "OneCycleLR", "CyclicLR"]:
          scheduler.step()
        elif scheduler_type == "StepLR":
            if (epoch + 1) % scheduler.step_size == 0:
                scheduler.step()
                # StepLR is typically called after a certain number of epochs (step_size)
        elif scheduler_type == "ReduceLROnPlateau":
            pass # because we are in the training loop and so no validation dataset applied

        print()

    time_elapsed = time.time() - since
    print(f'Training is completed in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best train Acc: {best_acc:4f}')  # Based on training accuracy

    model.load_state_dict(best_model_wts)
    return model, train_acc_history, train_loss_history

In [ ]:
class_names = ['Benign', 'Malignant', 'Normal']

In [ ]:
# Modified Train_Model_Final_KFold function
def Train_Model_Final_KFold(model, criterion, optimizer, Dataset_New, num_epochs, scheduler, scheduler_type, K):

    since = time.time()
    # best_model_wts = copy.deepcopy(model.state_dict()) # We might not need to save a "best" model here in the same way, as each fold trains a model on the full dataset
    # best_loss = float('inf') # Not tracking validation loss for early stopping here
    train_acc_history = [] # Still useful to see training progress
    train_loss_history = [] # Still useful to see training progress

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(str(device))

    kf = KFold(n_splits=K, shuffle=True, random_state=42)

    # List to store the trained models from each fold
    trained_models = []

    # We will iterate K times, but use the entire Dataset_New for training each time
    # The KFold split is used conceptually here to iterate K times, but the indices are not used to create subsets for training.
    for fold in range(K):
        print(f"Training Fold {fold + 1}/{K}")

        # Create a DataLoader for the entire Dataset_New for training
        # Ensure the Dataset_New has the training transform applied
        Dataset_New.dataset.transform = Train_transform

        def my_collate_fn(batch):
            images = []
            labels = []

            for image, label in batch:
                image = transforms.ToPILImage()(image)  # Convert to PIL Image
                image = transforms.Resize((224, 224))(image)  # Add resize transform here
                image = transforms.ToTensor()(image)  # Convert to Tensor using ToTensor
                images.append(image)
                labels.append(label)

            # Stack images into a batch tensor
            images = torch.stack(images, dim=0)
            labels = torch.tensor(labels)
            return images, labels

        train_loader = DataLoader(Dataset_New, batch_size=best_params_for_k["batch_size"], shuffle=True, num_workers = 32, collate_fn = my_collate_fn)

        # Create a new model instance for each fold to ensure fresh weights
        fold_model, fold_criterion, fold_optimizer = Model_ResNet101_PyTorch(learning_rate=best_params_for_k["learning_rate"],
                                                                              dropout_rate= best_params_for_k["dropout_rate"])

        fold_model.to(str(device))
        # Re-initialize the scheduler for each fold's optimizer
        # If using Optuna, you would need pass best scheduler type and its params here.
        scheduler_fold = create_scheduler(
             fold_optimizer, # leave as it is it automatically picks from the scheduler being used
             scheduler_type, # Leave as it is, it automatically picks from the scheduler being used
           max_lr=0.005433329251927817, epochs=86, steps_per_epoch=19) # specify this value per-scheduler type used manually for each run


        # Train the model on the entire Dataset_New
        trained_model_fold, fold_train_acc_history, fold_train_loss_history = Train_Model_Final(fold_model, fold_criterion, fold_optimizer,
                                                                                                train_loader, num_epochs, scheduler_fold, scheduler_type)

        # Store the trained model for later evaluation
        trained_models.append(copy.deepcopy(trained_model_fold.state_dict()))

    time_elapsed = time.time() - since
    print(f'Total training time for {K} folds: {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    # Return the list of trained model state dictionaries
    return trained_models

In [ ]:
# This Gap is left to separate the two Training_Model_Final_KFolds

In [ ]:
# Train K models, each on the entire Dataset_New
trained_model_states = Train_Model_Final_KFold(model, criterion, optimizer, Dataset_New,
                                               num_epochs = best_params_for_k["num_epochs"],
                                               scheduler = scheduler,
                                               scheduler_type = scheduler_types, # Pass the scheduler type
                                               K= K) # gamma1 is not used in the training loop here

Training Fold 1/10
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-14

In [ ]:
#import numpy as np
#from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
#import torch

def evaluate_model_for_k(trained_model_states, Test_dataloader, K, class_names):

    fold_metrics = []
    all_probs_accumulated = [] # To store probabilities for AUC calculation

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    for fold, model_state_dict in enumerate(trained_model_states):
        print(f"Evaluating Fold {fold + 1}/{K}")

        # Create a new model instance and load the trained weights for this fold
        model, _, _ = Model_ResNet101_PyTorch(learning_rate=best_params_for_k["learning_rate"],
                                              dropout_rate= best_params_for_k["dropout_rate"])

        model.load_state_dict(model_state_dict)
        model.to(device)
        model.eval()

        all_preds_fold = []
        all_labels_fold = []
        all_probs_fold = [] # To store probabilities for this fold

        with torch.no_grad():  # Disable gradient calculation during evaluation
            for inputs, labels in Test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                probs = outputs # Determines the probabilities used for calculating the auc score values
                _, preds = torch.max(outputs, 1)
                all_preds_fold.extend(preds.cpu().numpy())
                all_labels_fold.extend(labels.cpu().numpy())

                all_probs_fold.extend(probs.cpu().numpy())

        all_probs_accumulated.extend(all_probs_fold)

        # Calculate overall metrics for this fold's evaluation
        metrics_this_fold = {}
        metrics_this_fold['accuracy'] = accuracy_score(all_labels_fold, all_preds_fold)
        metrics_this_fold['precision'] = precision_score(all_labels_fold, all_preds_fold, average='macro', zero_division=0)
        metrics_this_fold['recall'] = recall_score(all_labels_fold,all_preds_fold, average='macro', zero_division=0) # Should be all_labels_fold
        metrics_this_fold['f1-score'] = f1_score(all_labels_fold, all_preds_fold, average='macro', zero_division=0)

        # Calculate per-class metrics for this fold's evaluation
        per_class_metrics_this_fold = {}
        cm_this_fold = confusion_matrix(all_labels_fold, all_preds_fold)

        for i, class_name in enumerate(class_names):
            # Handle potential errors if a class is not present in the fold's labels/predictions
            tp = cm_this_fold[i, i] if i < cm_this_fold.shape[0] and i < cm_this_fold.shape[1] else 0

            # Ensure column and row sums are within bounds
            fp = np.sum(cm_this_fold[:, i]) - tp if i < cm_this_fold.shape[1] else 0
            fn = np.sum(cm_this_fold[i, :]) - tp if i < cm_this_fold.shape[0] else 0

            # Calculate precision, recall, and f1-score for the class
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

            per_class_metrics_this_fold[class_name] = {
                'precision': precision,
                'recall': recall,
                'f1-score': f1
            }

        fold_metrics.append({'overall': metrics_this_fold, 'per_class': per_class_metrics_this_fold})

    # Calculate mean and standard deviation of overall metrics across all folds
    overall_metrics_across_folds = {metric: [f['overall'][metric] for f in fold_metrics if not np.isnan(f['overall'][metric])] for metric in ['accuracy', 'precision', 'recall', 'f1-score']} # Removed 'auc'

    mean_overall_metrics = {metric: np.mean(values) if values else np.nan for metric, values in overall_metrics_across_folds.items()}
    std_overall_metrics = {metric: np.std(values) if values else np.nan for metric, values in overall_metrics_across_folds.items()}


    # Calculate average per-class metrics across all folds
    per_class_metrics_across_folds = {
        class_name: {
            metric: [fold_data['per_class'][class_name][metric] for fold_data in fold_metrics]
            for metric in ['precision', 'recall', 'f1-score']
        }
        for class_name in class_names
    }

    mean_per_class_metrics = {
        class_name: {
            metric: np.mean(values)
            for metric, values in metrics.items()
        }
        for class_name, metrics in per_class_metrics_across_folds.items()
    }

    std_per_class_metrics = {
        class_name: {
            metric: np.std(values)
            for metric, values in metrics.items()
        }
        for class_name, metrics in per_class_metrics_across_folds.items()
    }


    return {
        'fold_metrics': fold_metrics,
        'mean_overall_metrics': mean_overall_metrics,
        'std_overall_metrics': std_overall_metrics,
        'mean_per_class_metrics': mean_per_class_metrics, # Add mean per-class metrics
        'std_per_class_metrics': std_per_class_metrics # Add standard deviation of per-class metrics
        }

In [ ]:
# Evaluate each of the K trained models on the Test_dataloader
metrics_k = evaluate_model_for_k(trained_model_states, Test_dataloader, K, class_names)

Evaluating Fold 1/10
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,096
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]          16,384
      BatchNorm2d-12          [-1, 256, 56, 56]             512
           Conv2d-13          [-1, 256, 56, 56]          16,384
      BatchNorm2d-

In [ ]:
# Print the evaluation results
print("Metrics per Fold (evaluated on Test_dataset):")
for fold_num, fold_data in enumerate(metrics_k['fold_metrics']):
    print(f"\n--- Fold {fold_num + 1} ---")
    print("  Overall Metrics for this Fold:")
    for metric_name, value in fold_data['overall'].items():
        print(f"    {metric_name.replace('_', ' ').title()}: {value}")

    print("  Per-Class Metrics for this Fold:")
    for class_name, metrics in fold_data['per_class'].items():
        print(f"    Class {class_name}:")
        print(f"      Precision: {metrics['precision']}")
        print(f"      Recall: {metrics['recall']}")
        print(f"      F1-score: {metrics['f1-score']}")


print("\n--- Mean Overall Metrics Across All Folds (evaluated on Test_dataset) ---")
for metric_name, value in metrics_k['mean_overall_metrics'].items():
     print(f"  Mean {metric_name.replace('_', ' ').title()}: {value}")

print("\n--- Standard Deviation Overall Metrics Across All Folds (evaluated on Test_dataset) ---")
for metric_name, value in metrics_k['std_overall_metrics'].items():
     print(f"  Std Dev {metric_name.replace('_', ' ').title()}: {value}")

print("\n--- Mean Per-Class Metrics Across All Folds (evaluated on Test_dataset) ---")
for class_name, metrics in metrics_k['mean_per_class_metrics'].items():
    print(f"  Class {class_name}:")
    print(f"    Mean Precision: {metrics['precision']}")
    print(f"    Mean Recall: {metrics['recall']}")
    print(f"    Mean F1-score: {metrics['f1-score']}")

print("\n--- Standard Deviation Per-Class Metrics Across All Folds (evaluated on Test_dataset) ---")
for class_name, metrics in metrics_k['std_per_class_metrics'].items():
    print(f"  Class {class_name}:")
    print(f"    Std Dev Precision: {metrics['precision']}")
    print(f"    Std Dev Recall: {metrics['recall']}")
    print(f"    Std Dev F1-score: {metrics['f1-score']}")

Metrics per Fold (evaluated on Test_dataset):

--- Fold 1 ---
  Overall Metrics for this Fold:
    Accuracy: 0.8189655172413793
    Precision: 0.7946081504702195
    Recall: 0.8267195767195767
    F1-Score: 0.8080175424048942
  Per-Class Metrics for this Fold:
    Class Benign:
      Precision: 0.896551724137931
      Recall: 0.8253968253968254
      F1-score: 0.8595041322314049
    Class Malignant:
      Precision: 0.7272727272727273
      Recall: 0.75
      F1-score: 0.7384615384615384
    Class Normal:
      Precision: 0.76
      Recall: 0.9047619047619048
      F1-score: 0.8260869565217391

--- Fold 2 ---
  Overall Metrics for this Fold:
    Accuracy: 0.8275862068965517
    Precision: 0.8618954248366012
    Recall: 0.7794312169312169
    F1-Score: 0.8100868112309759
  Per-Class Metrics for this Fold:
    Class Benign:
      Precision: 0.7866666666666666
      Recall: 0.9365079365079365
      F1-score: 0.8550724637681159
    Class Malignant:
      Precision: 0.9166666666666666
     